### Collect relevant paths

In [86]:
import re
import glob
from aicsimageio import AICSImage
import numpy as np
import cv2
from scipy import ndimage
from PIL import Image
import os
import re
from natsort import natsorted
import matplotlib.pyplot as plt
from skimage import io, img_as_float32, transform

split_ratio = 0.8
seed = 33
np.random.seed(seed)

In [87]:
datasets_to_augment = ["Study_3", "Study_4_1", "Study_4_2", "Study_5", "Study_6", "Study_7", "Study_17_1", "Study_17_2", 
                       "Study_17_3", "Study_17_4", "Study_17_5", "Study_17_6", "Study_20", "Study_21", "Study_24", "Study_29_1", "Study_29_2", "Study_29_3",
                       "Study_29_4",  "Study_30"]       #Studies with Actin and Tubulin

In [88]:
path = '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/'
img_files = glob.glob(path + '*z*')
ch1_files = glob.glob(path + '*Nuc*')
ch2_files = glob.glob(path + '*Mito*')
ch3_files = glob.glob(path + '*Actin*')
ch4_files = glob.glob(path + '*Tub*')

In [89]:
#def tile_image(image, patch_size=(256, 256), overlap=(0, 0)):
#    if image is None:
#        print("Error: Input image is None")
#        return
#
#    if len(image.shape) != 3:
#        print("Error: Input image should be a 3-dimensional array (height x width x channels)")
#        return
#        
#    height, width = image.shape[-2:]
#    patch_height, patch_width = patch_size
#    overlap_height, overlap_width = overlap

#    patches = []

#    # Calculate the number of patches in both dimensions
#    num_patches_height = (height - overlap_height) // (patch_height - overlap_height) + 1
#    num_patches_width = (width - overlap_width) // (patch_width - overlap_width) + 1

#    # Iterate over each patch position
#    for i in range(num_patches_height):
#        for j in range(num_patches_width):
#            # Calculate patch start and end positions
            
            
#            start_h = i * (patch_height - overlap_height)
#            end_h = min(start_h + patch_height, height)
#            if(end_h - start_h < 256):
#                start_h = (end_h - 256)
            
            
#            start_w = j * (patch_width - overlap_width)
#            end_w = min(start_w + patch_width, width)
#            if(end_w - start_w < 256):
#                start_w = (end_w - 256)
            

#            # Extract the patch
#            patch = image[:,start_h:end_h, start_w:end_w]

#            # Append the patch to the list
#            patches.append(patch)

#    return patches

In [90]:
def natural_sort_key(s):
    """Key function for natural sorting."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def min_max_normalize(image):
    min_val = np.min(image)
    max_val = np.max(image)
    normalized_image = (image - min_val) / (max_val - min_val)
    return normalized_image
    
def z_score_normalize(image):
    mean_val = np.mean(image)
    std_val = np.std(image)
    normalized_image = (image - mean_val) / std_val
    return normalized_image

def percentile_normalization(image, pmin=2, pmax=99.8, axis=None, dtype=np.uint16 ):
    '''
    Compute a percentile normalization for the given image.

    Parameters:
    - image (array): array of the image file.
    - pmin  (int or float): the minimal percentage for the percentiles to compute. 
                            Values must be between 0 and 100 inclusive.
    - pmax  (int or float): the maximal percentage for the percentiles to compute. 
                            Values must be between 0 and 100 inclusive.
    - axis : Axis or axes along which the percentiles are computed. 
             The default (=None) is to compute it along a flattened version of the array.
    - dtype (dtype): type of the wanted percentiles (uint16 by default)

    Returns:
    Normalized image (np.ndarray): An array containing the normalized image.
    '''

    if not (np.isscalar(pmin) and np.isscalar(pmax) and 0 <= pmin < pmax <= 100 ):
        raise ValueError("Invalid values for pmin and pmax")

    low_p  = np.percentile(image, pmin, axis=axis, keepdims=True)
    high_p = np.percentile(image, pmax, axis=axis, keepdims=True)

    if low_p == high_p:
        img_norm = image
        print(f"Same min {low_p} and high {high_p}, image may be empty")

    else:
        dtype_max = np.iinfo(dtype).max
        img_norm = dtype_max * (image - low_p) / (high_p - low_p)
        img_norm = img_norm.astype(dtype)

    return img_norm

In [91]:
img_files = sorted(img_files, key=natural_sort_key)
ch1_files = sorted(ch1_files, key=natural_sort_key)
ch2_files = sorted(ch2_files, key=natural_sort_key)
ch3_files = sorted(ch3_files, key=natural_sort_key)
ch4_files = sorted(ch4_files, key=natural_sort_key)

In [92]:
for file in img_files:
    print(file)

/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z0.ome.tiff
/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z1.ome.tiff
/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z2.ome.tiff
/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z3.ome.tiff
/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z4.ome.tiff
/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z5.ome.tiff
/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z6.ome.tiff
/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z7.ome.tiff
/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z8.ome.tiff
/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_253_DIC_z0.ome.tiff
/media/loc

In [93]:
print(f"Image files: {len(img_files)}")
print(f"Nuc files: {len(ch1_files)}")
print(f"Mito files: {len(ch2_files)}")
print(f"Actin files: {len(ch3_files)}")
print(f"Tub files: {len(ch4_files)}")

Image files: 207
Nuc files: 23
Mito files: 23
Actin files: 0
Tub files: 0


### Identify missing targets from channels

In [94]:
channel_lists = [ch1_files, ch2_files, ch3_files, ch4_files]  

In [95]:
all_channels = []

for channel in channel_lists:
    all_channels.extend(channel)

In [96]:
all_channels

['/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_Nucleus.ome.tiff',
 '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_253_Nucleus.ome.tiff',
 '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_254_Nucleus.ome.tiff',
 '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_255_Nucleus.ome.tiff',
 '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_256_Nucleus.ome.tiff',
 '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_257_Nucleus.ome.tiff',
 '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_258_Nucleus.ome.tiff',
 '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_259_Nucleus.ome.tiff',
 '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_260_Nucleus.ome.tiff',
 '/media/local-admin/galaxy/lightmycells_storage/images/Study_DI

In [97]:
# Example list of image filenames
target_filenames = all_channels

# Regular expression pattern to match 'image_xy'
pattern = r"image_\d+"

# Extract unique 'image_xy' identifiers
unique_target_identifiers = natsorted(set(re.findall(pattern, ' '.join(target_filenames))))

max_target_count = len(unique_target_identifiers)

# Print unique 'image_xy' identifiers
print("Unique target identifiers:")
for identifier in unique_target_identifiers:
    print(identifier)
print(max_target_count)

Unique target identifiers:
image_252
image_253
image_254
image_255
image_256
image_257
image_258
image_259
image_260
image_261
image_262
image_263
image_264
image_265
image_266
image_267
image_268
image_269
image_270
image_271
image_272
image_273
image_274
23


In [98]:
channel_names = ['Nuc', 'Mito', 'Actin', 'Tub']
missing_index_dict = {}

for i,channel in enumerate(channel_lists):
    if len(channel) < max_target_count:
        # Example list of image filenames
        image_filenames = img_files
    
        # Example list of target filenames
        target_filenames = channel
        
        # Regular expression pattern to match 'image_xy'
        pattern = r"image_\d+"
        
        # Extract unique 'image_xy' identifiers from image filenames
        unique_image_identifiers = set(re.findall(pattern, ' '.join(image_filenames)))
        
        # Extract 'image_xy' identifiers from target filenames
        target_image_identifiers = set(re.findall(pattern, ' '.join(target_filenames)))
        
        # List to store identifiers not present in target filenames
        missing_identifiers = []
        
        # Check if unique 'image_xy' identifiers are present in target filenames
        for identifier in unique_image_identifiers:
            if identifier not in target_image_identifiers:
                missing_identifiers.append(identifier)
        
        print(f"Identifiers not present in target filenames for {channel_names[i]}:", missing_identifiers)
        print(f"Number of missing items: {len(missing_identifiers)}")
    
        missing_indexes = []
        #for identifier in missing_identifiers:
        for item in missing_identifiers:
            index_of_missing = unique_target_identifiers.index(item)
            missing_indexes.append(index_of_missing)
            print(f'Index of missing item: {index_of_missing}')
        missing_index_dict[channel_names[i]] = sorted(missing_indexes)

Identifiers not present in target filenames for Actin: ['image_274', 'image_253', 'image_264', 'image_263', 'image_271', 'image_262', 'image_273', 'image_254', 'image_256', 'image_269', 'image_270', 'image_252', 'image_267', 'image_260', 'image_257', 'image_272', 'image_259', 'image_261', 'image_265', 'image_266', 'image_268', 'image_258', 'image_255']
Number of missing items: 23
Index of missing item: 22
Index of missing item: 1
Index of missing item: 12
Index of missing item: 11
Index of missing item: 19
Index of missing item: 10
Index of missing item: 21
Index of missing item: 2
Index of missing item: 4
Index of missing item: 17
Index of missing item: 18
Index of missing item: 0
Index of missing item: 15
Index of missing item: 8
Index of missing item: 5
Index of missing item: 20
Index of missing item: 7
Index of missing item: 9
Index of missing item: 13
Index of missing item: 14
Index of missing item: 16
Index of missing item: 6
Index of missing item: 3
Identifiers not present in ta

In [99]:
missing_index_dict

{'Actin': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22],
 'Tub': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22]}

### Collecting images

In [100]:
#for file in img_files:
    #img = AICSImage(file)
    #img_array = img.data
    #print(img_array.shape)

In [101]:
#unique_shapes = set()

#for file in img_files:
#    img = AICSImage(file)
#    img_array = img.data
#    shape = img_array.shape
#    unique_shapes.add(shape)

## Print the unique shapes
#for shape in unique_shapes:
    #print(shape)

In [102]:
# Dictionary to store filenames organized by image number
image_number_dict = {}

# Regular expression pattern to extract image numbers
pattern = r'image_(\d+)_\w+_z\d+\.ome\.tiff'

for filename in img_files:
    # Extract image number from filename
    match = re.search(pattern, filename)
    if match:
        image_number = int(match.group(1))
        # Add filename to the corresponding list based on image number
        if image_number in image_number_dict:
            image_number_dict[image_number].append(filename)
        else:
            image_number_dict[image_number] = [filename]

In [103]:
image_number_dict

{252: ['/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z0.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z1.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z2.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z3.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z4.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z5.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z6.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z7.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z8.ome.tiff'],
 253: ['/media/local-admin/galaxy/lightmycells_storage/ima

In [104]:
# Regular expression pattern to extract image numbers
#pattern = r'image_(\d+)_\w+.ome\.tiff'

#for channel in channel_lists:
#    for filename in channel:
#        # Extract image number from filename
#        match = re.search(pattern, filename)
#        if match:
#            image_number = int(match.group(1))
#            # Add filename to the corresponding list based on image number
#            if image_number in image_number_dict:
#                image_number_dict[image_number].append(filename)
#            else:
#                image_number_dict[image_number] = [filename]

In [105]:
# Iterate through the dictionary and print the number of items for each key
num_layers = []
for key, value in image_number_dict.items():
    num_layers.append(len(value))
    print(f"Number of items assigned to key {key}: {len(value)}")

Number of items assigned to key 252: 9
Number of items assigned to key 253: 9
Number of items assigned to key 254: 9
Number of items assigned to key 255: 9
Number of items assigned to key 256: 9
Number of items assigned to key 257: 9
Number of items assigned to key 258: 9
Number of items assigned to key 259: 9
Number of items assigned to key 260: 9
Number of items assigned to key 261: 9
Number of items assigned to key 262: 9
Number of items assigned to key 263: 9
Number of items assigned to key 264: 9
Number of items assigned to key 265: 9
Number of items assigned to key 266: 9
Number of items assigned to key 267: 9
Number of items assigned to key 268: 9
Number of items assigned to key 269: 9
Number of items assigned to key 270: 9
Number of items assigned to key 271: 9
Number of items assigned to key 272: 9
Number of items assigned to key 273: 9
Number of items assigned to key 274: 9


In [106]:
image_number_dict

{252: ['/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z0.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z1.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z2.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z3.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z4.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z5.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z6.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z7.ome.tiff',
  '/media/local-admin/galaxy/lightmycells_storage/images/Study_DIC/Study_18/image_252_DIC_z8.ome.tiff'],
 253: ['/media/local-admin/galaxy/lightmycells_storage/ima

In [108]:
img_data = []

for image_number, filenames in image_number_dict.items():
    
    # Iterate through the list of filenames for the current image number
    for filename in filenames:
        # Load image data using AICSImage
        img = AICSImage(filename)
        img_array = img.data
        #img_array = img_as_float32(img_array)
        img_array = np.squeeze(img_array)
        #img_array = cv2.resize(img_array, dsize=(resize_size[1], resize_size[0]), interpolation=cv2.INTER_CUBIC)
        #img_array = min_max_normalize(img_array)
        #img_array = z_score_normalize(img_array)
        #img_array = percentile_normalization(img_array)     !!!
        img_array = np.squeeze(img_array)
        # Append the image array to the list of image arrays for the current image number
        img_data.append(img_array)

In [109]:
img_array = np.array(img_data)
img_array = np.expand_dims(img_array, axis=1)

In [110]:
img_array.shape

(207, 1, 1200, 1200)

In [111]:
img_array.dtype

dtype('uint16')

In [112]:
len(img_array)

207

### Adding blank targets if they are missing

In [113]:
mask_stacks = []                                               #This is so if targets are missing, blanks are added in their place

for i,channel in enumerate(channel_lists):
    
    channel_list = []
    for mask in channel:
        mask_img = AICSImage(mask)
        mask_array = mask_img.data
        #mask_array = img_as_float32(mask_array)
        print(mask_array.dtype)
        #mask_array = np.squeeze(img_array)
        mask_array = np.squeeze(mask_array)
        #mask_array = cv2.resize(mask_array, dsize=resize_size)
        #mask_array = min_max_normalize(mask_array)                                     #change to float32, 
        #mask_array = z_score_normalize(mask_array)
        #mask_array = percentile_normalization(mask_array)
        channel_list.append(mask_array) 

    if channel_names[i] in missing_index_dict:
        #print(channel_names[i])
        # Get the indexes from the dictionary corresponding to the appropriate key
        zeros_array = np.zeros(img_array[0][0].shape).astype(np.uint16)                                         #!!!!!!!!!!!!!!!!!!!!!
        mito_indexes = missing_index_dict.get(channel_names[i], [])
        #print(mito_indexes)

        # Insert zeros_array at the indexes specified by mito_indexes in ch1_list
        for idx in mito_indexes:
            channel_list.insert(idx, zeros_array)
        
    mask_stacks.append(channel_list)

uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16


In [114]:
missing_index_dict

{'Actin': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22],
 'Tub': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22]}

In [115]:
for channel in mask_stacks:
    for i,x in enumerate(channel):
        if x.all() == 0:
            print(f'{i}: Replaced with blank')
    print(' ')

 
 
0: Replaced with blank
1: Replaced with blank
2: Replaced with blank
3: Replaced with blank
4: Replaced with blank
5: Replaced with blank
6: Replaced with blank
7: Replaced with blank
8: Replaced with blank
9: Replaced with blank
10: Replaced with blank
11: Replaced with blank
12: Replaced with blank
13: Replaced with blank
14: Replaced with blank
15: Replaced with blank
16: Replaced with blank
17: Replaced with blank
18: Replaced with blank
19: Replaced with blank
20: Replaced with blank
21: Replaced with blank
22: Replaced with blank
 
0: Replaced with blank
1: Replaced with blank
2: Replaced with blank
3: Replaced with blank
4: Replaced with blank
5: Replaced with blank
6: Replaced with blank
7: Replaced with blank
8: Replaced with blank
9: Replaced with blank
10: Replaced with blank
11: Replaced with blank
12: Replaced with blank
13: Replaced with blank
14: Replaced with blank
15: Replaced with blank
16: Replaced with blank
17: Replaced with blank
18: Replaced with blank
19: Re

#### Replicating masks for each image

In [116]:
mask_data = np.array(mask_stacks)

In [117]:
mask_data.dtype

dtype('uint16')

In [118]:
mask_data = np.swapaxes(mask_data, 0,1)

In [119]:
mask_data.dtype

dtype('uint16')

In [120]:
mask_data.shape

(23, 4, 1200, 1200)

In [121]:
num_layers

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]

In [122]:
# Repeat each element in the first dimension of mask_data based on num_layers
extended_mask_data = np.repeat(mask_data, num_layers, axis=0)

# Verify the shape of the extended array
print("Shape of extended mask_data:", extended_mask_data.shape)

Shape of extended mask_data: (207, 4, 1200, 1200)


In [123]:
extended_mask_data.dtype

dtype('uint16')

In [124]:
mask_data = extended_mask_data

In [125]:
img_data = img_array

In [126]:
img_data[0].shape

(1, 1200, 1200)

In [127]:
mask_data[0].shape

(4, 1200, 1200)

In [128]:
mask_data.shape

(207, 4, 1200, 1200)

In [129]:
mask_data.dtype

dtype('uint16')

In [130]:
img_data.dtype

dtype('uint16')

### Shuffle the "stacks" so it's not always the last x% that gets split off (while respecting the stacks)

In [131]:
# Initialize lists to store the split data
split_img_data = []
split_mask_data = []

# Iterate over num_layers to split img_data and mask_data
start_idx = 0
for num in num_layers:
    end_idx = start_idx + num
    split_img_data.append(img_data[start_idx:end_idx])
    split_mask_data.append(mask_data[start_idx:end_idx])
    start_idx = end_idx

# Verify the split data
print("Data split according to stacks before shuffling:")
for i, (img, mask) in enumerate(zip(split_img_data, split_mask_data)):
    print(f"Stack {i+1}: Image data shape: {img.shape}, Mask data shape: {mask.shape}, Num layers: {num_layers[i]}")

 #Shuffle the data
shuffled_indices = np.random.permutation(len(split_img_data))
shuffled_img_data = [split_img_data[i] for i in shuffled_indices]
shuffled_mask_data = [split_mask_data[i] for i in shuffled_indices]
shuffled_num_layers = [num_layers[i] for i in shuffled_indices]

# Verify the shuffled data
print("Data split according to stacks after shuffling:")
for i, (img, mask) in enumerate(zip(shuffled_img_data, shuffled_mask_data)):
    print(f"Stack {i+1}: Image data shape: {img.shape}, Mask data shape: {mask.shape}, Num layers: {shuffled_num_layers[i]}")

# Reunite the shuffled data into a single array
reunited_shuffled_img_data = np.concatenate(shuffled_img_data, axis=0)
reunited_shuffled_mask_data = np.concatenate(shuffled_mask_data, axis=0)

# Verify the shape of the reunited shuffled data
print("Reunited Shuffled Image Data Shape:", reunited_shuffled_img_data.shape)
print("Reunited Shuffled Mask Data Shape:", reunited_shuffled_mask_data.shape)

img_data = reunited_shuffled_img_data
mask_data = reunited_shuffled_mask_data
num_layers = shuffled_num_layers

Data split according to stacks before shuffling:
Stack 1: Image data shape: (9, 1, 1200, 1200), Mask data shape: (9, 4, 1200, 1200), Num layers: 9
Stack 2: Image data shape: (9, 1, 1200, 1200), Mask data shape: (9, 4, 1200, 1200), Num layers: 9
Stack 3: Image data shape: (9, 1, 1200, 1200), Mask data shape: (9, 4, 1200, 1200), Num layers: 9
Stack 4: Image data shape: (9, 1, 1200, 1200), Mask data shape: (9, 4, 1200, 1200), Num layers: 9
Stack 5: Image data shape: (9, 1, 1200, 1200), Mask data shape: (9, 4, 1200, 1200), Num layers: 9
Stack 6: Image data shape: (9, 1, 1200, 1200), Mask data shape: (9, 4, 1200, 1200), Num layers: 9
Stack 7: Image data shape: (9, 1, 1200, 1200), Mask data shape: (9, 4, 1200, 1200), Num layers: 9
Stack 8: Image data shape: (9, 1, 1200, 1200), Mask data shape: (9, 4, 1200, 1200), Num layers: 9
Stack 9: Image data shape: (9, 1, 1200, 1200), Mask data shape: (9, 4, 1200, 1200), Num layers: 9
Stack 10: Image data shape: (9, 1, 1200, 1200), Mask data shape: (9, 

In [132]:
img_data.dtype

dtype('uint16')

In [133]:
mask_data.dtype

dtype('uint16')

### Split into train, valid, test

#### Split train and test_t

In [134]:
# Example data
num_stacks = len(num_layers)

# Accumulate the count of images in each stack
stack_indices = np.cumsum(num_layers)

# Total number of images
total_images = sum(num_layers)

# Calculate the split indices
split_index = int(total_images * split_ratio)  # Assuming 80% training and 20% testing, change this if you have more data

# Find the stack index at which the split occurs
stack_index = np.argmax(stack_indices >= split_index)

if ((num_stacks-1)-stack_index) < 2:                 # So there is at least 1 stack separated (which also means that the test data will be empty. Should this be 2 instead? But then ratio of data could be very different. Would rather have empty valid than test!!!             
    stack_index = stack_index-1
    print(stack_index)

index = stack_indices[stack_index]   #???

# Split the data into training and testing sets
train_data = img_data[:index]
train_targets = mask_data[:index]
test_data_t = img_data[index:]
test_targets_t = mask_data[index:]

# Print the shapes of the resulting datasets
print("Num stacks:", num_stacks)
print("Stack indices:", stack_indices)
print("Split index:", split_index)
print("Stack index:", stack_index)
print("Index:", index)
print("Training Image Data Shape:", train_data.shape)
print("Training Image Data Shape:", train_data.shape)
print("Training Mask Data Shape:", train_targets.shape)
print("Testing Image Data Shape:", test_data_t.shape)
print("Testing Mask Data Shape:", test_targets_t.shape)

Num stacks: 23
Stack indices: [  9  18  27  36  45  54  63  72  81  90  99 108 117 126 135 144 153 162
 171 180 189 198 207]
Split index: 165
Stack index: 18
Index: 171
Training Image Data Shape: (171, 1, 1200, 1200)
Training Image Data Shape: (171, 1, 1200, 1200)
Training Mask Data Shape: (171, 4, 1200, 1200)
Testing Image Data Shape: (36, 1, 1200, 1200)
Testing Mask Data Shape: (36, 4, 1200, 1200)


In [135]:
## Iterate through each image in train_targets
#for train_image_idx, train_image in enumerate(train_targets):
#    # Iterate through each image in test_targets_t
#    for test_image_idx, test_image in enumerate(test_targets_t):
#        # Check if the current train image is identical to the current test image
#        if np.array_equal(train_image, test_image):
#            print(f"Duplicate found! Train image index: {train_image_idx}, Test image index: {test_image_idx}")

In [136]:
train_data.dtype

dtype('uint16')

In [137]:
train_targets.dtype

dtype('uint16')

In [138]:
test_data_t.dtype

dtype('uint16')

In [139]:
test_targets_t.dtype

dtype('uint16')

#### Split into test and valid

In [140]:
num_layers_test = num_layers[stack_index+1:]

In [141]:
num_layers_test

[9, 9, 9, 9]

In [142]:
num_stacks_test = len(num_layers_test)

# Accumulate the count of images in each stack
stack_indices_test = np.cumsum(num_layers_test)

# Total number of images
total_images_test = sum(num_layers_test)

# Calculate the split indices
split_index_test = int(total_images_test * 0.5)  # Assuming 80% training and 20% testing

# Find the stack index at which the split occurs
stack_index_test = np.argmax(stack_indices_test >= split_index_test)
    
index_test = stack_indices_test[stack_index_test]
print(index_test)

# Split the data into valid and testing sets
validation_data = test_data_t[:index_test]
validation_targets = test_targets_t[:index_test]
test_data = test_data_t[index_test:]
test_targets = test_targets_t[index_test:]

# Print the shapes of the resulting datasets
print("Valid Image Data Shape:", validation_data.shape)
print("Valid Mask Data Shape:", validation_targets.shape)
print("Test Image Data Shape:", test_data.shape)
print("Test Mask Data Shape:", test_targets.shape)

18
Valid Image Data Shape: (18, 1, 1200, 1200)
Valid Mask Data Shape: (18, 4, 1200, 1200)
Test Image Data Shape: (18, 1, 1200, 1200)
Test Mask Data Shape: (18, 4, 1200, 1200)


In [143]:
## Iterate through each image in train_targets
#for train_image_idx, train_image in enumerate(train_targets):
#    # Iterate through each image in test_targets_t
#    for test_image_idx, test_image in enumerate(test_targets):
#        # Check if the current train image is identical to the current test image
#        if np.array_equal(train_image, test_image):
#            print(f"Duplicate found! Train image index: {train_image_idx}, Test image index: {test_image_idx}")

In [144]:
validation_data.dtype

dtype('uint16')

In [145]:
validation_targets.dtype

dtype('uint16')

In [146]:
test_data.dtype

dtype('uint16')

In [147]:
test_targets.dtype

dtype('uint16')

### Resizing


In [148]:
resize_size = (1024, 1024)

In [149]:
train_data.shape

(171, 1, 1200, 1200)

In [150]:
resized_train_data = []

# Iterate through each image in the train_data
for img in train_data:
    resized_slices = []
    # img is currently of shape (C, 1200, 1200)
    for slice_idx in range(img.shape[0]):
        # Resize each slice individually
        img_slice_resized = cv2.resize(img[slice_idx], dsize=(1024, 1024), interpolation=cv2.INTER_CUBIC)
        resized_slices.append(img_slice_resized)
    
    # Stack the resized slices along the first dimension
    img_resized = np.stack(resized_slices, axis=0)

    # Append the resized image to the new array
    resized_train_data.append(img_resized)

# Convert the list to a numpy array
resized_train_data = np.array(resized_train_data)

# Verify the shape
print(resized_train_data.shape)


(171, 1, 1024, 1024)


In [151]:
resized_train_targets = []

# Iterate through each mask in the train_targets
for mask in train_targets:
    resized_slices = []
    # mask is currently of shape (C, 1200, 1200)
    for slice_idx in range(mask.shape[0]):
        # Resize each slice individually
        mask_slice_resized = cv2.resize(mask[slice_idx], dsize=(1024, 1024), interpolation=cv2.INTER_CUBIC)
        resized_slices.append(mask_slice_resized)
    
    # Stack the resized slices along the first dimension
    mask_resized = np.stack(resized_slices, axis=0)

    # Append the resized mask to the new array
    resized_train_targets.append(mask_resized)

# Convert the list to a numpy array
resized_train_targets = np.array(resized_train_targets)

# Verify the shape
print(resized_train_targets.shape)

(171, 4, 1024, 1024)


In [152]:
resized_valid_data = []
resized_valid_targets = []

# Iterate through each image and mask in the valid_data and valid_targets
for img, mask in zip(validation_data, validation_targets):
    # Resize the image
    resized_img_slices = []
    for slice_idx in range(img.shape[0]):
        img_slice_resized = cv2.resize(img[slice_idx], dsize=(1024, 1024), interpolation=cv2.INTER_CUBIC)
        resized_img_slices.append(img_slice_resized)
    img_resized = np.stack(resized_img_slices, axis=0)
    resized_valid_data.append(img_resized)
    
    # Resize the mask
    resized_mask_slices = []
    for slice_idx in range(mask.shape[0]):
        mask_slice_resized = cv2.resize(mask[slice_idx], dsize=(1024, 1024), interpolation=cv2.INTER_CUBIC)
        resized_mask_slices.append(mask_slice_resized)
    mask_resized = np.stack(resized_mask_slices, axis=0)
    resized_valid_targets.append(mask_resized)

# Convert the lists to numpy arrays
resized_valid_data = np.array(resized_valid_data)
resized_valid_targets = np.array(resized_valid_targets)

# Verify the shapes
print(resized_valid_data.shape)
print(resized_valid_targets.shape)

(18, 1, 1024, 1024)
(18, 4, 1024, 1024)


In [153]:
train_data = resized_train_data 
train_targets = resized_train_targets
validation_data = resized_valid_data
validation_targets =resized_valid_targets

### Data augmentation

In [154]:
def augment_images(train_images, target_images, augmentation_percentage=0.3):
    augmented_train_images = []
    augmented_target_images = []
    num_images_to_augment = int(len(train_images) * augmentation_percentage)
    indices_to_augment = np.random.choice(len(train_images), size=num_images_to_augment, replace=False)
    
    for idx, (train_img, target_img) in enumerate(zip(train_images, target_images)):
        if idx in indices_to_augment:
            augmented_train_img = train_img.copy()
            augmented_target_img = target_img.copy()
            random_var = np.random.rand()
            noise = np.random.normal(loc=0, scale=1, size=augmented_train_img[0].shape).astype(np.uint16)
            print(type(noise))
            print(augmented_train_img.dtype)
            print(augmented_target_img.dtype)
            
            for layer in range(augmented_train_img.shape[0]):                             
                # Apply flipping & noise
                if random_var <= 0.2:
                    augmented_train_img[layer] = np.flipud(augmented_train_img[layer])      # All of them will never happen together, right?
                elif 0.2 <= random_var <= 0.8:
                   augmented_train_img[layer] = np.clip(augmented_train_img[layer] + noise, 0, 65535)
                else:
                    augmented_train_img[layer] = np.fliplr(augmented_train_img[layer])       

            augmented_train_images.append(augmented_train_img)
            
            for layer in range(augmented_target_img.shape[0]):
                if np.all(augmented_target_img[layer] == 0):
                    continue
                    
                elif random_var <= 0.2:
                    augmented_target_img[layer] = np.flipud(augmented_target_img[layer])
                elif 0.2 <= random_var <= 0.8:
                    augmented_target_img[layer] = np.clip(augmented_target_img[layer] + noise, 0, 65535)
                else:
                    augmented_target_img[layer] = np.fliplr(augmented_target_img[layer])
                 
            augmented_target_images.append(augmented_target_img)

            print(augmented_train_img.dtype)
            print(augmented_target_img.dtype)

        
        
        augmented_train_images.append(train_img)          # keep the originals too
        augmented_target_images.append(target_img)
        print(train_img.dtype)
        print(target_img.dtype)
        
        
    
    return np.array(augmented_train_images), np.array(augmented_target_images)

In [155]:
present_tub = 0                             #In study_DIC_110, tubulin is only present in 3 datasets, so I want to augment those
for target in train_targets:
    if np.any(target[3] != 0):
        present_tub += 1
print(f"Number of masks with tubulin before agumentation: {present_tub}")

Number of masks with tubulin before agumentation: 0


In [156]:
augmented_train_data = train_data
augmented_target_data = train_targets
for dataset in datasets_to_augment:
    if dataset in path:
        augmented_train_data, augmented_target_data = augment_images(train_data, train_targets, augmentation_percentage=0.5)
        break

In [157]:
present_tub = 0
for target in augmented_target_data:
    if np.any(target[3] != 0):
        present_tub += 1
print(f"Number of masks with tubulin after agumentation: {present_tub}")

Number of masks with tubulin after agumentation: 0


In [158]:
print(augmented_train_data.shape)
print(augmented_target_data.shape)

(171, 1, 1024, 1024)
(171, 4, 1024, 1024)


In [159]:
test_data.shape

(18, 1, 1200, 1200)

In [160]:
train_data = augmented_train_data
train_targets = augmented_target_data

In [161]:
print(train_data.dtype)
print(train_targets.dtype)

uint16
uint16


### Pytable

##### Create pytable (run only once)

See create_pytables.ipynb

### Adding data to pytable

In [162]:
import tables
import numpy as np

In [163]:
#def batch_write_to_pytables(data_list, table, batch_size=1000):
#    total_records = len(data_list)
#    for i in range(0, total_records, batch_size):
#        batch_data = data_list[i:i+batch_size]
#        table.append(batch_data)
#        table.flush()

#### Add train dataset

In [164]:
# Open an existing PyTable file or create a new one
with tables.open_file('/media/local-admin/lightmycells/thesis/code/pytables/single/DIC/study_DIC_train_single.pytable', mode='a') as h5file:

    # Access the desired group or table
    img_earray = h5file.root.img.data
    mask_earray = h5file.root.mask.data

    for record in train_data:
        record = np.expand_dims(record, axis=0)
        img_earray.append(record)

    for record in train_targets:
        record = np.expand_dims(record, axis=0)
        mask_earray.append(record)  

#### Add validation dataset

In [165]:
# Open an existing PyTable file or create a new one
with tables.open_file('/media/local-admin/lightmycells/thesis/code/pytables/single/DIC/study_DIC_valid_single.pytable', mode='a') as h5file:

    # Access the desired group or table
    img_earray = h5file.root.img.data
    mask_earray = h5file.root.mask.data

    for record in validation_data:
        record = np.expand_dims(record, axis=0)
        img_earray.append(record)

    for record in validation_targets:
        record = np.expand_dims(record, axis=0)
        mask_earray.append(record)  

#### Add test dataset

In [166]:
# Open an existing PyTable file or create a new one
with tables.open_file('/media/local-admin/lightmycells/thesis/code/pytables/single/DIC/study_DIC_test_single.pytable', mode='a') as h5file:

    # Access the desired group or table

    data_shape = test_data.shape
    shape_suffix = f"{data_shape[-2]}"  # Assuming the shape you're interested in is in the second-to-last dimension
    
    # Access the desired group or table
    img_earray = h5file.root.img[f'data_{shape_suffix}']
    mask_earray = h5file.root.mask[f'data_{shape_suffix}']
    
    # Iterate through your data and append it to the EArray
    for data_row in test_data:
        data_row = np.expand_dims(data_row, axis=0)
        img_earray.append(data_row)
        print(data_row.dtype)

    for data_row in test_targets:
        data_row = np.expand_dims(data_row, axis=0)
        mask_earray.append(data_row)
        print(data_row.dtype)

uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16


#### Check pytable structures

In [167]:
train_file = '/media/local-admin/lightmycells/thesis/code/pytables/single/DIC/study_DIC_train_single.pytable'
test_file = '/media/local-admin/lightmycells/thesis/code/pytables/single/DIC/study_DIC_test_single.pytable'
valid_file = '/media/local-admin/lightmycells/thesis/code/pytables/single/DIC/study_DIC_valid_single.pytable'

# Open the PyTable file
with tables.open_file(test_file, mode='r') as h5file:
    # Print the structure of the file
    print(h5file)

    # Access and print information about groups
    for group in h5file.walk_groups():
        print(f"Group: {group}")

    # Access and print information about arrays
    for array in h5file.walk_nodes(where='/', classname='Array'):
        print(f"Array: {array}")

/media/local-admin/lightmycells/thesis/code/pytables/study_DIC_test_single.pytable (File) ''
Last modif.: '2024-06-17T13:04:02+00:00'
Object Tree: 
/ (RootGroup) ''
/img (Group) 'Image data'
/img/data_1024 (EArray(0, 1, 1024, 1024)zlib(5)) ''
/img/data_1200 (EArray(18, 1, 1200, 1200)zlib(5)) ''
/img/data_1300 (EArray(0, 1, 1300, 1624)zlib(5)) ''
/img/data_2044 (EArray(0, 1, 2044, 2048)zlib(5)) ''
/img/data_2048 (EArray(0, 1, 2048, 2048)zlib(5)) ''
/img/data_512 (EArray(0, 1, 512, 512)zlib(5)) ''
/img/data_966 (EArray(0, 1, 966, 1296)zlib(5)) ''
/img/data_980 (EArray(0, 1, 980, 1016)zlib(5)) ''
/mask (Group) 'Mask data'
/mask/data_1024 (EArray(0, 4, 1024, 1024)zlib(5)) ''
/mask/data_1200 (EArray(18, 4, 1200, 1200)zlib(5)) ''
/mask/data_1300 (EArray(0, 4, 1300, 1624)zlib(5)) ''
/mask/data_2044 (EArray(0, 4, 2044, 2048)zlib(5)) ''
/mask/data_2048 (EArray(0, 4, 2048, 2048)zlib(5)) ''
/mask/data_512 (EArray(0, 4, 512, 512)zlib(5)) ''
/mask/data_966 (EArray(0, 4, 966, 1296)zlib(5)) ''
/mask/

In [168]:
train_targets.shape

(171, 4, 1024, 1024)

In [169]:
train_data.shape

(171, 1, 1024, 1024)

In [170]:
tables.file._open_files.close_all()